## HW4 LSH
106062314 蔡政諺

In [1]:
import re
import math
from random import randint
from pyspark import SparkConf, SparkContext

In [2]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("LSH")
sc = SparkContext(conf=conf)
sc.setCheckpointDir("./checkpoints")

### Shingling
這一步要將 ```./athletics``` 中的 101 篇文章從 txt 檔轉換成 **Documents x Shingles** 的 key-value pairs。
#### mapper1
對於每篇文章，將文章根據空白字元 ```" "``` 切開，接著刪除每個字詞前後的標點符號，並過濾掉空字串。
#### mapper2
定義一個全域變數 ```idx```， 每一個字詞生成 ```(idx+i, word)``` 的 ```kShingle``` 個 key-value pairs。<br>
E.g., ```"Claxton"``` → ```[(0, "Claxton"), (1, "Claxton"), (2, "Claxton")]```
#### reducer1
將 ```mapper2``` 生出的 kay-value pairs 中，key 相同的接起來，形成 k-shingles。<br>
E.g., ```(2, "Claxton"), (2, "hunting"), (2, "first")``` → ```(2, "Claxton hunting first")```
#### mapper3
將 ```reducer1``` 生出的 kay-value pairs 的 value 當作新的 key，文章編號當作新的 value。<br>
E.g., ```(2, "Claxton hunting first")``` → ```("Claxton hunting first", [1])```
#### reducer2
將同一篇文章中重複出現的 shingle 移除。<br>
E.g., ```("Claxton hunting first", [1]), ("Claxton hunting first", [1])``` → ```("Claxton hunting first", [1])```
#### union
將所有文章的 shingle 的集合存至 ```shingles```。
#### reducer3
將同一個 shingle 的所有 key-value pairs 組合，新的 value 為所有包含該 shingle 的文章編號。<br>
此時用 ```N``` 記錄 shingle 的總數。<br>
E.g., ```("Claxton hunting first", [1]), ("Claxton hunting first", [2])``` → ```("Claxton hunting first", [1, 2])```
#### mapper4
用全域變數 ```idx``` 將所有的 shingle 字串替換成編號，則得到 Shingles x Documents 的 key-value pairs。<br>
E.g., ```("Claxton hunting first", [1, 2])``` → ```(0, [1, 2])```
#### mapper5
將 Shingles x Documents 轉換為 Documents x Shingles。<br>
E.g., ```(0, [1, 2])``` → ```[(1, [0]), (2, [0])]```<br>
接著再使用 ```reducer3``` 將同一個 document 的 shingles 組合，即完成 shingling 的流程。

In [3]:
kShingle = 3
filenum = 101

def mapper1(line):
    line = line.split(" ")
    words = []
    for word in line:
        for i in range(4):
            word = re.sub('[^a-zA-Z0-9]$', '', word)
            word = re.sub('^[^a-zA-Z0-9]', '', word)
        if word != "":
            words.append(word)
    return words

idx = -1
def mapper2(word):
    global idx
    idx += 1
    list = []
    for i in range(kShingle):
        list.append((idx+i, word))
    return list

def reducer1(x, y):
    return x + " " + y

def mapper3(x, i):
    return (x[1], [i])

def reducer2(x, y):
    return x

def reducer3(x, y):
    return x + y

def mapper4(x):
    global idx
    idx += 1
    return (x[1], idx)

def mapper5(x):
    list = []
    for i in range(len(x[0])):
        list.append((x[0][i], [x[1]]))
    return list

In [4]:
shingles = sc.emptyRDD()
for i in range(1, filenum+1):
    words = sc.textFile("./athletics/%03d.txt" % (i)).flatMap(mapper1)
    words = words.flatMap(mapper2)
    words = words.reduceByKey(reducer1)
    words = words.map(lambda x: mapper3(x, i)).reduceByKey(reducer2)
    shingles = shingles.union(words).repartition(4)
    shingles.checkpoint()
shingles = shingles.reduceByKey(reducer3)
N = shingles.count()
shingles = shingles.map(mapper4).flatMap(mapper5).reduceByKey(reducer3).sortBy(lambda x: x[0])
print(shingles.collect())

[(1, [31, 195, 280, 321, 611, 975, 1181, 1525, 1667, 1788, 1889, 1977, 2286, 2294, 2315, 2511, 2813, 2825, 2941, 3026, 3083, 3130, 3401, 3437, 3539, 3783, 3849, 3876, 4045, 4067, 4143, 4197, 4307, 4377, 4542, 4592, 4601, 4642, 4892, 5161, 5210, 5374, 5502, 5726, 5761, 5884, 5893, 5999, 6308, 6334, 6371, 6440, 6598, 29, 156, 291, 315, 419, 612, 656, 666, 669, 869, 998, 1039, 1052, 1185, 1358, 1514, 1584, 1598, 1600, 1610, 1611, 1707, 1880, 1898, 1933, 1983, 1985, 2010, 2130, 2632, 2753, 2764, 2894, 3021, 3223, 3377, 3390, 3595, 3801, 4342, 4637, 4672, 4812, 4935, 5042, 5071, 5084, 5212, 5696, 5728, 5854, 5898, 5967, 6090, 6150, 6188, 6318, 6428, 6471, 100, 249, 310, 612, 668, 888, 1426, 1428, 1545, 1853, 1873, 2160, 2273, 2278, 2474, 2657, 2840, 2914, 3024, 3080, 3436, 3492, 3673, 3773, 4070, 4238, 4485, 4562, 4567, 4764, 4781, 4845, 4846, 4893, 4910, 5173, 5284, 5363, 5878, 6006, 6358, 6396, 6513, 74, 164, 169, 307, 375, 376, 380, 613, 666, 667, 674, 720, 862, 1453, 1634, 1674, 1963, 1

### Min-Hashing
這一步要將 **Documents x Shingles** 的 key-value pairs 轉換成 **Documents x Signatures** 的 key-value pairs。
#### isPrime
判斷 ```num``` 是不是質數。
#### getMinPrime
回傳大於 ```N``` 的最小質數，用來當作 ```p```。
#### generateRandomHash
生成一組 random hash function 的係數 ```a``` 和 ```b```。<br>
```a``` 是小於 ```N``` 的正整數。<br>
```b``` 是小於 ```N``` 的非負整數。
#### hashing
回傳 $((ax + b) \ mod \ p) \ mod \ N$ 的結果。
#### mapper6
對於每個 hash function $h_{\pi}$，依序計算 $h_{\pi}(0), h_{\pi}(1), ..., h_{\pi}(p-1)$ 的值。<br>
對於所有 hash value $h_{\pi}(i)$，若 document 中有編號 $h_{\pi}(i)$ 的 shingle，便將最小的 ```i``` 記錄，即 $h_{\pi}(C) = min_{\pi} \pi (C)$。<br>
E.g., ```(1, [1, 3, 4])``` → ```(1, [1, 2])```

In [5]:
numOfHashFunctions = 100

def isPrime(num):
    for i in range(2, math.floor(math.sqrt(num))+1):
        if num % i == 0:
            return False
    return True

def getMinPrime():
    i = N
    while(True):
        i += 1
        if isPrime(i):
            return i

def generateRandomHash():
    a = randint(1, N-1)
    b = randint(0, N-1)
    return [a, b]

def hashing(x, a, b):
    return ((a * x + b) % p) % N
    
def mapper6(x):
    list = []
    for func in hashFunctions:
        for i in range(p):
            if hashing(x=i, a=func[0], b=func[1]) in x[1]:
                list.append(i)
                break
    return (x[0], list)

In [6]:
p = getMinPrime()
print("N:", N)
print("p:", p)
hashFunctions = []
for i in range(numOfHashFunctions):
    hashFunctions.append(generateRandomHash())
signatures = shingles.map(lambda x: mapper6(x)).sortBy(lambda x: x[0])
print(signatures.collect())

N: 26901
p: 26903
[(1, [123, 116, 58, 22, 203, 45, 5, 79, 158, 24, 559, 499, 137, 115, 15, 202, 161, 51, 153, 21, 194, 149, 36, 55, 47, 161, 43, 198, 25, 52, 6, 23, 164, 95, 121, 317, 8, 162, 20, 32, 37, 84, 49, 1, 51, 44, 355, 63, 65, 135, 155, 228, 73, 102, 130, 184, 40, 36, 90, 73, 89, 367, 14, 34, 0, 70, 67, 182, 166, 72, 97, 127, 44, 23, 190, 49, 113, 302, 135, 32, 30, 56, 138, 62, 10, 12, 40, 32, 146, 55, 4, 40, 38, 64, 53, 67, 328, 11, 81, 55]), (2, [88, 251, 36, 551, 36, 22, 84, 18, 183, 71, 12, 491, 62, 10, 64, 25, 235, 398, 19, 8, 55, 254, 66, 75, 251, 222, 82, 12, 442, 302, 93, 145, 29, 36, 36, 464, 39, 188, 629, 633, 331, 303, 103, 320, 496, 4, 184, 741, 106, 112, 615, 258, 240, 130, 39, 297, 224, 49, 18, 55, 245, 136, 329, 367, 61, 212, 479, 120, 105, 114, 112, 412, 151, 232, 189, 50, 456, 397, 8, 172, 423, 29, 238, 547, 102, 18, 16, 158, 351, 105, 172, 73, 36, 122, 26, 375, 6, 198, 119, 1]), (3, [91, 64, 20, 52, 164, 185, 186, 23, 130, 171, 26, 70, 14, 10, 66, 299, 143, 1

### Locality-Sensitive Hashing (LSH)
這一步要從 **Documents x Signatures** 的 key-value pairs 中找出 **candidate pairs**。
#### LSH
根據 band 的大小，生成一個 ```r``` 個 argument 的 random hash function。<br>
此處由於 ```r = 2```，LSH 會是 $h(x_1, x_2) = (c_1x_1 + c_2x_2) \ mod \ k$。
#### mapper7
將 signatures 以 band 為單位，hash 到 ```numOfBuckets``` 個 bucket 裡。<br>
E.g., ```(1, [1, 0, 0, 0, 0, 0, 1, 1, 0, 2])``` → ```(1, [5, 0, 0, 6, 3])```
#### cartesian
將兩兩文章組合起來。
#### mapper8
兩兩文章比較是否有 hash 到同一個 bucket 的 band。若有，代表它是 candidate pair，將其回傳。<br>
E.g., ```(1, [5, 0, 0, 6, 3]), (2, [1, 1, 1, 1, 3])``` → ```[(1, 2)]```<br>
E.g., ```(1, [5, 0, 0, 6, 3]), (2, [1, 1, 1, 1, 1])``` → ```[]```

In [7]:
r = 2
b = 50
numOfBuckets = 10000

coef = []
for i in range(r):
    val = randint(1, numOfBuckets-1)
    coef.append(val)
    
def LSH(band):
    val = 0
    for i in range(r):
        val += (coef[i] * band[i]) % numOfBuckets
    val = val % numOfBuckets
    return val

def mapper7(x):
    list = []
    for i in range(b):
        bucket = LSH(x[1][i*r : (i+1)*r])
        list.append(bucket)
    return (x[0], list)

def mapper8(x):
    for i in range(b):
        if x[0][1][i] == x[1][1][i]:
            if x[0][0] < x[1][0]:
                return [(x[0][0], x[1][0])]
    return []

In [8]:
bands = signatures.map(mapper7)
candidatePairs = bands.cartesian(bands).flatMap(mapper8)
print(candidatePairs.collect())

[(1, 9), (8, 13), (12, 20), (14, 23), (17, 23), (1, 27), (1, 41), (3, 30), (7, 24), (7, 25), (8, 24), (15, 24), (14, 27), (8, 47), (9, 40), (14, 40), (14, 49), (15, 44), (18, 30), (16, 33), (18, 35), (23, 38), (17, 48), (23, 39), (23, 47), (23, 48), (23, 49), (3, 61), (4, 65), (14, 62), (9, 72), (10, 69), (10, 73), (17, 54), (17, 58), (18, 58), (22, 58), (22, 61), (23, 62), (17, 70), (18, 70), (3, 76), (3, 81), (9, 81), (13, 88), (14, 86), (8, 100), (9, 99), (14, 96), (17, 82), (23, 82), (17, 89), (25, 43), (30, 34), (30, 35), (30, 37), (30, 38), (27, 40), (29, 44), (30, 47), (35, 37), (35, 38), (34, 40), (35, 47), (36, 43), (37, 39), (38, 46), (39, 46), (47, 48), (47, 49), (48, 49), (28, 55), (30, 58), (30, 61), (35, 55), (34, 58), (35, 58), (35, 61), (34, 66), (37, 68), (50, 51), (42, 57), (44, 55), (44, 57), (48, 54), (40, 59), (40, 69), (47, 66), (47, 69), (48, 66), (48, 69), (49, 66), (49, 69), (24, 77), (24, 85), (29, 97), (33, 81), (34, 88), (37, 88), (32, 94), (35, 97), (36, 92

### Calculate Similarity
計算 **candidate pairs** 的 **Jaccard similarity**。
#### join
將 ```candidatePairs``` 與 ```shingles``` 中 key 相同的 key-value pairs 組合在一起。<br>
目的是讓 ```candidatePairs``` 裡有相對應文章的兩篇文章的 shingles。
#### mapper9
將第一次 ```join``` 的結果的 key 改為 candidate pair 的後項，這樣下一次 ```join``` 就能得到後項的 shingles。<br>
E.g., ```(1, (2, [1, 3, 4]))``` → ```(2, (1, [1, 3, 4]))```
#### mapper10
將第二次 ```join``` 的結果重新排列。<br>
E.g., ```(2, ((1, [1, 3, 4]), [2, 3, 5]))``` → ```((1, 2), ([1, 3, 4], [2, 3, 5]))```
#### mapper11
對 ```mapper10``` 生出的 kay-value pairs 的兩組 shingles 算 Jaccard similarity，當作新的 value。<br>
E.g., ```((1, 2), ([1, 3, 4], [2, 3, 5]))``` → ```((1, 2), 0.2)```

In [9]:
def mapper9(x):
    return (x[1][0], (x[0], x[1][1]))

def mapper10(x):
    return ((x[1][0][0], x[0]), (x[1][0][1], x[1][1]))

def mapper11(x):
    similarity = len(set(x[1][0]) & set(x[1][1])) / len(set(x[1][0]) | set(x[1][1])) * 100
    return (x[0], similarity)

candidatePairs = candidatePairs.join(shingles).map(mapper9).join(shingles).map(mapper10)
candidatePairs = candidatePairs.map(mapper11).sortBy(lambda x: (-x[1], x[0][0]))

### Results
將 Jaccard similarity 最高的 10 組 pairs 輸出。

In [10]:
results = candidatePairs.take(10)
for result in results:
    print("(%03d, %03d) : %.2f%s" % (result[0][0], result[0][1], result[1], "%"))

(012, 020) : 100.00%
(052, 084) : 100.00%
(047, 049) : 76.09%
(030, 035) : 71.75%
(049, 088) : 52.59%
(048, 049) : 50.06%
(023, 038) : 48.92%
(014, 040) : 42.13%
(047, 088) : 40.06%
(047, 048) : 38.20%
